# 08-01-Milestone-Project-2


Assignment: Create a Complete BlackJack Card Game in Python.

Here are the requirements:

- You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
- The game needs to have one player versus an automated dealer.
- The player can stand or hit.
- The player must be able to pick their betting amount.
- You need to keep track of the player's total money.
- You need to alert the player of wins, losses, or busts, etc...
- You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand.


In [8]:
# imports
from IPython.display import clear_output
import random
import os

# Definitions
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven',
        'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7,
        'Eight': 8, 'Nine': 9, 'Ten': 10, 'Jack': 10, 'Queen': 10, 'King': 10, 'Ace': 11}

# classes

class Deck:

    def __init__(self):
        self.all_cards = []
        for suit in suits:
            for rank in ranks:
                self.all_cards.append(Card(suit, rank))

    def shuffle(self):
        random.shuffle(self.all_cards)

    def deal(self):
        return self.all_cards.pop()

    def __str__(self):
        card_list = [str(card) for card in self.all_cards]
        return str(card_list)

class Card:

    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        return self.rank + ' of ' + self.suit


class Chips:

    def __init__(self):
        self.total = 100  # This can be set to a default value or supplied by a user input
        self.bet = 0

    def win_bet(self):
        self.total += 2*self.bet
        self.bet = 0

    def lose_bet(self):
        self.total -= self.bet
        self.bet = 0

    def __str__(self):
        return "The Player has " + str(self.total) + " chips"


class Table:
    def __init__(self):
        self.cards = []  # start with an empty list as we did in the Deck class
        self.value = 0   # start with zero value
        self.aces = 0    # add an attribute to keep track of aces

    def add_cards(self, card):
        self.value += values[card.rank]
        self.cards.append(card)
        if card.rank == "Ace":
            self.aces += 1

    def adjust_for_ace(self):
        if self.value >= 22 and self.aces != 0:
            self.value = self.value - self.aces * 10

    def parcial_table(self):
        card_list = [str(card) for card in self.cards]
        parcial_card_list = card_list[1:]
        return "The Dealer has the faceup cards " + str(parcial_card_list)

    def __str__(self):
        card_list = [str(card) for card in self.cards]
        return "The Dealer has in the hand " + str(card_list) + " with sums " + str(self.value)

    def clear_table(self):
        self.cards = []
        self.value = 0
        self.aces = 0


class Hand:
    def __init__(self):
        self.cards = []  # start with an empty list as we did in the Deck class
        self.value = 0   # start with zero value
        self.aces = 0    # add an attribute to keep track of aces

    def add_cards(self, card):
        self.value += values[card.rank]
        self.cards.append(card)
        if card.rank == "Ace":
            self.aces += 1

    def adjust_for_ace(self):
        while self.value > 21 and self.aces > 0:
            self.value -= 10
            self.aces -= 1

    def __str__(self):
        card_list = [str(card) for card in self.cards]
        return "The Player has in the hand " + str(card_list) + " with sums " + str(self.value)

    def clear_hand(self):
        self.cards = []
        self.value = 0
        self.aces = 0

# functions

def take_bet(chip_class_object):
    chip_class_object.bet = 0
    val = 0
    while True:
        try:
            val = int(input("Please enter your bet: "))
            if val > chip_class_object.total:
                clear_screen()
                print("You do not have enough chips!")
                continue
            elif val <= chip_class_object.total:
                chip_class_object.bet = val
                break
            break
        except:
            clear_screen()
            print("Looks like you did not enter an integer!")
            continue
    clear_screen()
    print("Your bet is " + str(chip_class_object.bet))


def hit(deck_class_object, hand_class_object):
    hand_class_object.add_cards(deck_class_object.deal())
    hand_class_object.adjust_for_ace()


def hit_or_stand(deck, hand):
    global playing  # to control an upcoming while loop

    while True:
        x = input("Would you like to Hit or Stand? Enter 'h' or 's' ")

        if x[0].lower() == 'h':
            hit(deck, hand)
            
        elif x[0].lower() == 's':
            print("Player stands. Dealer is playing.")
            playing = False

        else:
            print("Sorry, please try again.")
            continue
        break


def show_some(player_hand, dealer_hand):
    print(player_hand)
    print(dealer_hand.parcial_table())

def show_all(player, dealer):
    print(player_hand)
    print(dealer_hand)

def player_busts(player_hand, dealer_hand, player_chips):
    print("Player Busted!")
    player_chips.lose_bet()

def player_wins(player_hand, dealer_hand, player_chips):
    print("Player Wins!")
    player_chips.win_bet()

def dealer_busts(player_hand, dealer_hand, player_chips):
    print("Player Wins! Dealer Busted!")
    player_chips.win_bet()

def dealer_wins(player_hand, dealer_hand, player_chips):
    print("Dealer Wins!")
    player_chips.lose_bet()

def push():
    print("Dealer and player tie! PUSH!!")

def clear_screen():
    '''
    Function that clears the output, uses different function depending on the environment
    '''
    try:
        # Check if running in IPython environment
        ipython = get_ipython()
        if ipython is not None:
            # Execute in IPython environment
            clear_output(wait=True)
    except NameError:
        # Execute in Python terminal
        os.system('cls' if os.name == 'nt' else 'clear')

if __name__ == '__main__':
    
    # Create & shuffle the deck, deal two cards to each player
    # Player hand and Dealer table
    player_hand = Hand()
    dealer_hand = Table()

    # Playind deck and shuffle
    deck = Deck()
    deck.shuffle()

    # Set up the Player's chips
    player_chips = Chips()

    # Print an opening statement
    print("This is a game of Blackjack.")
    
    # Game logic
    while True:

        play_game = input('Are you ready to play? Enter Yes or No.')

        if play_game.lower()[0] == 'y':
            playing = True
        else:
            playing = False
            break

        # clear_screen()

        # Prompt the Player for their bet
        print("You have " + str(player_chips.total) + " chips")

        take_bet(player_chips)

        # Show cards (but keep one dealer card hidden)
        player_hand.clear_hand()
        dealer_hand.clear_table()
        player_hand.add_cards(deck.deal())
        player_hand.add_cards(deck.deal())
        dealer_hand.add_cards(deck.deal())
        dealer_hand.add_cards(deck.deal())
        show_some(player_hand, dealer_hand)

        while playing:  # recall this variable from our hit_or_stand function

            # Prompt for Player to Hit or Stand
            hit_or_stand(deck, player_hand)

            # Show cards (but keep one dealer card hidden)
            show_some(player_hand, dealer_hand)

            # If player's hand exceeds 21, run player_busts() and break out of loop
            if player_hand.value > 21:
                player_busts(player_hand, dealer_hand, player_chips)
                break

        # If Player hasn't busted, play Dealer's hand until Dealer reaches 17
        if player_hand.value <= 21:

            while dealer_hand.value < 17:
                hit(deck, dealer_hand)

            # Show all cards
            show_all(player_hand, dealer_hand)

            # Run different winning scenarios

            if dealer_hand.value > 21:
                dealer_busts(player_hand, dealer_hand, player_chips)

            elif dealer_hand.value > player_hand.value:
                dealer_wins(player_hand, dealer_hand, player_chips)

            elif dealer_hand.value < player_hand.value:
                player_wins(player_hand, dealer_hand, player_chips)

            else:
                push(player_hand, dealer_hand)

        # Inform Player of their chips total
        print("\nPlayer's winnings stand at", player_chips.total)

        # Ask to play again

        if player_chips.total == 0:
            print("You dont have more money!")
            break
        else:
            play_game = input(
                'Do you wany to play another round? Enter Yes or No.')

            if play_game.lower()[0] == 'y':
                playing = True
                continue
            else:
                print("Thank you for playing!")
                break

Your bet is 40
The Player has in the hand ['Three of Spades', 'Seven of Diamonds'] with sums 10
The Dealer has the faceup cards ['Three of Clubs']
The Player has in the hand ['Three of Spades', 'Seven of Diamonds', 'Ace of Clubs'] with sums 21
The Dealer has the faceup cards ['Three of Clubs']
The Player has in the hand ['Three of Spades', 'Seven of Diamonds', 'Ace of Clubs', 'Ace of Hearts'] with sums 12
The Dealer has the faceup cards ['Three of Clubs']
Player stands. Dealer is playing.
The Player has in the hand ['Three of Spades', 'Seven of Diamonds', 'Ace of Clubs', 'Ace of Hearts'] with sums 12
The Dealer has the faceup cards ['Three of Clubs']
The Player has in the hand ['Three of Spades', 'Seven of Diamonds', 'Ace of Clubs', 'Ace of Hearts'] with sums 12
The Dealer has in the hand ['Eight of Clubs', 'Three of Clubs', 'Queen of Hearts'] with sums 21
Dealer Wins!

Player's winnings stand at 60
Thank you for playing!
